In [6]:
import os
os.chdir('C:/Users/myhyu/OneDrive/Desktop/2024 미래에셋/InvestmentPropensity')

In [1]:
# 라이브러리 
import requests
import pandas as pd
import io
import time
import requests
from bs4 import BeautifulSoup
import numpy as np

In [44]:
# pip install requests beautifulsoup4 pandas


Note: you may need to restart the kernel to use updated packages.


## 네이버 증권 크롤링 함수

In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_financial_summary(stock_code):
    # 네이버 증권의 특정 종목 페이지 URL
    url = f'https://finance.naver.com/item/main.nhn?code={stock_code}'
    
    # 웹 페이지 요청
    response = requests.get(url)
    response.encoding = 'cp949'
    
    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Financial Summary 테이블 찾기
    table = soup.find('table', {'class': 'tb_type1 tb_num tb_type1_ifrs'})
    
    # 테이블 데이터 파싱
    df = pd.read_html(str(table))[0]
    
    # 데이터프레임 정리
    df.columns = df.columns.droplevel(0)  # 상위 MultiIndex 제거
    df.columns = df.columns.droplevel(1) 
    df.set_index(df.columns[0], inplace=True) # 첫 번째 열을 인덱스로 설정
    
    # 숫자 데이터로 변환할 열 선택 및 변환
    numeric_columns = ['2024.03']
    df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')
    
    # 매출 성장률 계산
    try:
        sales_growth = ((df.iloc[0, 8] - df.iloc[0, 7]) / df.iloc[0, 7]) * 100
    except KeyError:
        sales_growth = float('NaN')  # 매출액 데이터가 없는 경우 NaN 처리
    except np.core._exceptions.UFuncTypeError:
        sales_growth = float('NaN')  # 매출액 데이터가 없는 경우 NaN 처리
    
    # 매출성장률을 새로운 행으로 추가
    df.loc['매출성장률'] = sales_growth
    
    return df


In [ ]:
# 필요한 지표들 선택
    df = df.loc[['영업이익률', '순이익률', '매출액', '부채비율', 'PER(배)', 'PBR(배)', 'ROE(지배주주)', '매출성장률']]
    df=df[['2024.03']]
    
    return df

## 기업별 종목코드 불러오기

In [7]:
# 저장된 종목코드 불러오기
code_data = pd.read_csv('code.csv')
code_data = code_data[['종목코드']]

## 모든 기업별로 재무제표 합치기

In [8]:
# 재무제표 크롤링
for idx, firm_code in enumerate(code_data['종목코드']):
    print(idx + 1, '/', len(code_data['종목코드']))
    try:
        # 종목별 재무제표 크롤링
        try:
            sub_finance_table = get_financial_summary(firm_code)
        except requests.exceptions.Timeout:
            time.sleep(60)
            sub_finance_table = get_financial_summary(firm_code)

        # 종목별 재무제표 정리
        for num, col in enumerate(sub_finance_table.columns):
            temp = pd.DataFrame({firm_code : sub_finance_table[col]}).T
            temp.columns = [[col] * len(sub_finance_table), temp.columns]
            if num == 0:
                temp_arranged = temp
            else:
                temp_arranged = pd.merge(temp_arranged, temp, how='outer', left_index=True, right_index=True)

        # 재무제표 통합        
        if idx == 0:
            finance_table = temp_arranged
        else:
            finance_table = pd.concat([finance_table, temp_arranged])
    except ValueError:
        continue
    except KeyError:
        continue

# 재무제표 저장
finance_table.to_csv('finance_table.csv')

1 / 2696


C:\Users\myhyu\AppData\Local\Temp\ipykernel_34212\4065996145.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


2 / 2696


C:\Users\myhyu\AppData\Local\Temp\ipykernel_34212\4065996145.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


3 / 2696


...

C:\Users\myhyu\AppData\Local\Temp\ipykernel_34212\4065996145.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


129 / 2696


C:\Users\myhyu\AppData\Local\Temp\ipykernel_34212\4065996145.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


130 / 2696


KeyboardInterrupt: 

In [147]:
pd.read_csv('finance_table.csv')

,Unnamed: 0,2024.03,2024.03.1,2024.03.2,2024.03.3,2024.03.4,2024.03.5
0,주요재무정보,영업이익률,순이익률,매출액,부채비율,PER(배),PBR(배)
1,095570,7.95,2.16,2448.0,305.26,13.03,0.52
2,006840,8.08,3.56,11934.0,319.33,10.23,0.33
3,027410,9.55,9.71,1185.0,14.85,4.76,0.22
4,282330,1.67,1.2,19538.0,214.53,10.62,1.98
...,...,...,...,...,...,...,...
2616,024060,-0.59,0.41,306.0,17.3,179.8,1.84
2617,010240,9.2,9.77,353.0,25.14,5.56,0.62
2618,189980,8.2,3.72,225.0,112.21,12.28,0.91
2619,037440,3.17,5.63,563.0,169.13,13.1,1.19


In [24]:
import requests
from bs4 import BeautifulSoup

# URL 설정
url = 'https://finance.naver.com/item/coinfo.naver?code=035720'  # 카카오의 코드

# HTTP 요청 및 페이지 내용 가져오기
response = requests.get(url)
response.raise_for_status()  # 요청 실패 시 예외 발생

# BeautifulSoup을 사용하여 HTML 파싱
soup = BeautifulSoup(response.text, 'html.parser')

# EV/EBITDA 값 찾기
# 여기서는 실제 HTML 구조에 따라 적절한 선택자를 사용해야 합니다.
# HTML 구조를 확인 후 수정이 필요할 수 있습니다.

# 예시로 '//*[@id="tab_con1"]/table/tbody/tr[12]/td[2]'을 사용한 경우
# 이 XPath를 찾기 위해 브라우저의 개발자 도구를 사용하여 정확한 위치를 확인해야 합니다.
# 여기에 맞는 CSS Selector 또는 XPath를 사용합니다.

# EV/EBITDA 값을 찾기 위한 CSS 선택자 또는 XPath 사용 예시
# CSS Selector 예시 (테이블 구조에 따라 다를 수 있음)
ev_ebitda_element = soup.select_one('#tab_con1 table tbody tr:nth-of-type(12) td:nth-of-type(2)')

if ev_ebitda_element:
    ev_ebitda = ev_ebitda_element.get_text(strip=True)
    print('EV/EBITDA:', ev_ebitda)
else:
    print('EV/EBITDA 값을 찾을 수 없습니다.')



EV/EBITDA 값을 찾을 수 없습니다.


In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller

# Chrome 드라이버 자동 설치 및 설정
chromedriver_autoinstaller.install()
chrome_options = Options()
chrome_options.add_argument('--headless')  # GUI 없이 실행
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

service = Service()
driver = webdriver.Chrome(service=service, options=chrome_options)

# 웹 페이지 열기
url = 'https://finance.naver.com/item/coinfo.naver?code=035720'  # 삼성전자 예시
driver.get(url)

# 데이터 추출
try:
    ev_ebitda_element = driver.find_element(By.XPATH, '//td[contains(text(), "EV/EBITDA")]/following-sibling::td')
    ev_ebitda = ev_ebitda_element.text
    print('EV/EBITDA:', ev_ebitda)
finally:
    driver.quit()  # 브라우저 닫기


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//td[contains(text(), "EV/EBITDA")]/following-sibling::td"}
  (Session info: chrome-headless-shell=126.0.6478.182); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x006BC203+27395]
	(No symbol) [0x00653E04]
	(No symbol) [0x00551B7F]
	(No symbol) [0x00592C65]
	(No symbol) [0x00592D3B]
	(No symbol) [0x005CEC82]
	(No symbol) [0x005B39E4]
	(No symbol) [0x005CCB24]
	(No symbol) [0x005B3736]
	(No symbol) [0x00587541]
	(No symbol) [0x005880BD]
	GetHandleVerifier [0x00973AB3+2876339]
	GetHandleVerifier [0x009C7F7D+3221629]
	GetHandleVerifier [0x0073D674+556916]
	GetHandleVerifier [0x0074478C+585868]
	(No symbol) [0x0065CE44]
	(No symbol) [0x00659858]
	(No symbol) [0x006599F7]
	(No symbol) [0x0064BF4E]
	BaseThreadInitThunk [0x76207BA9+25]
	RtlInitializeExceptionC